In [ ]:
import pandas as pd
from NorenRestApiPy.NorenApi import  NorenApi
#import logging
from datetime import datetime
import datetime as dt
from pytz import timezone
import pandas_ta as ta
import broker 
import websocket
import os
import csv
import time
import math
import pyotp
from datetime import timedelta
pd.set_option('display.max_rows',None)

In [ ]:
user        = broker.user ##enter your usrid
pwd         = broker.pwd ##enter your password
vc          = broker.vc ##enter your vc
imei        = 'mac_address'

app_key     = # broker.get_appkey().rstrip("\n") ##your app key
time.sleep(2)

class ShoonyaApiPy(NorenApi):
    def __init__(self):
        NorenApi.__init__(self, host='https://api.shoonya.com/NorenWClientTP/', websocket='wss://api.shoonya.com/NorenWSTP/', eodhost='https://api.shoonya.com/chartApi/getdata/')
api = ShoonyaApiPy()

while True:
    try:
        factor2  = pyotp.TOTP(broker.key).now()
        ret = api.login(userid=user, password=pwd, twoFA=factor2, vendor_code=vc, api_secret=app_key, imei=imei)
        if ret['stat']=='Ok' : 
            print ('login successful')
            break
    except Exception:
        print('could not login, retrying')
        time.sleep(2)
        continue  


In [ ]:
#*********Download symbols.txt***************

fno_scrips = pd.read_csv('https://shoonya.finvasia.com/NFO_symbols.txt.zip',compression='zip', engine='python',delimiter=',')
fno_scrips['Expiry'] = pd.to_datetime(fno_scrips['Expiry'])
fno_scrips['StrikePrice'] = fno_scrips['StrikePrice'].astype(float)
fno_scrips.sort_values('Expiry',inplace=True)
fno_scrips.reset_index(drop=True, inplace=True)

In [ ]:
base=50
symbol="NIFTY"
otm_points=500
bias_points=-100
sl_factor=1.25

In [ ]:
def get_atm_strike():
    fut_token=str(fno_scrips[(fno_scrips['Instrument']=='FUTIDX') & (fno_scrips['Symbol']==symbol)].sort_values('Expiry').iloc[0]['Token']) 
    fut=float(api.get_quotes(exchange='NFO', token=fut_token)['lp'])
          
    atm_strike=round(fut/base)*base
    return atm_strike

def get_option_strikes() :
    atm_strike=get_atm_strike() 
    ce_strike=atm_strike+otm_points+bias_points
    pe_strike=atm_strike-otm_points+bias_points
    return(ce_strike,pe_strike)

#following func. returns all the scrip details , extract anything using key
def get_symbol(symbol,strike_price,optiontype):
    trading_symbol=fno_scrips[(fno_scrips['Symbol']==symbol) & (fno_scrips['OptionType']==optiontype) & (fno_scrips['StrikePrice']==strike_price)].iloc[0]
    return(trading_symbol)

def place_strangle(ce_trading_symbol,pe_trading_symbol) :
    
    ##placing orders*************************************************
    ce_order=api.place_order(buy_or_sell='S', product_type='M',
                        exchange='NFO', tradingsymbol=ce_trading_symbol, 
                        quantity=50, discloseqty=0,price_type='MKT', price=0, trigger_price=None,
                        retention='DAY', remarks='ce_order_001')
    pe_order=api.place_order(buy_or_sell='S', product_type='M',
                        exchange='NFO', tradingsymbol=pe_trading_symbol, 
                        quantity=50, discloseqty=0,price_type='MKT', price=0, trigger_price=None,
                        retention='DAY', remarks='pe_order_001')
    ce_order_id=ce_order['norenordno']
    pe_order_id=pe_order['norenordno']
    return(ce_order_id,pe_order_id)

def place_stop(tysm,sl_prc) :
    #define_trigg
    sl_trigg = sl_prc - 0.1
    sl_order=api.place_order(buy_or_sell='B', product_type='M',
                        exchange='NFO', tradingsymbol=tysm, 
                        quantity=50, discloseqty=0,price_type='SL-LMT', price=sl_prc, trigger_price=sl_trigg,
                        retention='DAY', remarks='sl_001')
    sl_order_id=sl_order['norenordno']
    return(sl_order_id)

    
def order_history(orderid,req): 
        while True: 
            json_data=api.single_order_history(orderid)
            if json_data!=None:
                break

        for id in json_data:
            value_return=id[req]
            break

        return value_return

def trail_sl(ce_sl_order_id,pe_sl_order_id):
    while True : 
        time.sleep(1)
        call_sl_status= order_history(ce_sl_order_id,'status')
        put_sl_status=order_history(pe_sl_order_id,'status')

        if call_sl_status=='COMPLETE':
            put_sl_to_cost = api.modify_order(exchange='NFO', tradingsymbol=pe_tsym, orderno=pe_sl_order_id,
                                 newquantity=50, newprice_type='SL-LMT', newprice=put_price, newtrigger_price=put_price-0.1)
            naked_leg_sym=pe_tsym
            break  

        if put_sl_status=='COMPLETE':
            call_sl_to_cost= api.modify_order(exchange='NFO', tradingsymbol=ce_tsym, orderno=ce_sl_order_id,
                                 newquantity=50, newprice_type='SL-LMT', newprice=call_price, newtrigger_price=call_price-0.1)
            naked_leg_sym=ce_tsym
            break
    return(naked_leg_sym)

    
    

In [ ]:
strikes = get_option_strikes()
ce_strike=strikes[0]
pe_strike=strikes[1]
##get symbols of PE & CE *****************************************
ce_tsym=get_symbol(symbol,ce_strike,"CE")['TradingSymbol'] 
pe_tsym=get_symbol(symbol,pe_strike,"PE")['TradingSymbol'] 

# ce_ltp=float(api.get_quotes(exchange='NFO', token=ce_token)['lp'])
# pe_ltp=float(api.get_quotes(exchange='NFO', token=pe_token)['lp'])

#ce_tsym,pe_tsym

In [ ]:
# #PLACE ORDERS

strangle_orders=place_strangle(ce_tsym,pe_tsym)


In [ ]:
#get status and avgprc

# call_status= order_history(strangle_orders[0],'status')
# put_status=order_history(strangle_orders[1],'status')

call_price=float(order_history(strangle_orders[0],'avgprc'))
put_price=float(order_history(strangle_orders[1],'avgprc'))
#define sl price
call_sl_price=round(call_price*sl_factor,1)
put_sl_price=round(put_price*sl_factor,1)
 


In [ ]:
ce_sl_order_id=place_stop(ce_tsym,call_sl_price)
pe_sl_order_id=place_stop(pe_tsym,put_sl_price)


In [ ]:
naked_tsym=trail_sl(ce_sl_order_id,pe_sl_order_id)